In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
from transformers import TextDataset,LineByLineTextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer



In [5]:
translated_dataset = pd.read_csv('translated_romance_2.csv')
translated_dataset

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,f3643541-c339-4377-851d-144ea400d551,Ibu tiri,Eric dan istrinya memiliki seorang putri berna...,Istri Eric meninggal.,Eric dan Meg sangat sedih.,Eric bertemu dengan seorang wanita dan menikah...,Meg sekarang senang dengan ibu tirinya yang baru.
1,d61b69c7-7eca-4099-ade2-1a7e5a014a13,Riasan yang buruk,Marnie akan menikah.,"Untuk pernikahannya, dia perlu makeup selesai.",Akibatnya dia menyewa penata rias untuk memban...,"Ketika penata rias selesai, Marinir melihatnya.","Dia ngeri dan akhirnya memecatnya,"
2,3ea38bec-1101-4a64-86fa-948904d33a2b,Bersatu kembali di reuni,Tom selalu bertanya -tanya apakah dia pernah m...,Dia telah memutuskan untuk menikahi orang lain...,"Sekarang, dia bercerai dan akan datang ke reun...",Tom dan Sue menghabiskan sebagian besar reuni ...,Kedua kekasih yang bersatu kembali telah bersa...
3,f085aab6-c13a-46d6-aca0-cf942b50577a,Keluarga baru,Jenn dan Bill jatuh cinta dan menikah.,"Beberapa bulan kemudian, Jenn mengetahui bahwa...",Bill sangat bersemangat dan tidak sabar untuk ...,Mereka mengetahui bahwa mereka memiliki bayi l...,"Begitu bayi itu lahir, mereka menamainya Theod..."
4,e19867fe-9b77-4a56-934a-9fd00d4c802e,Lone Girl,Dessie ingin bertemu pasangan baru.,Dia pergi melalui banyak situs kencan tidak be...,"Akhirnya, dia menyerah dan pergi ke perpustakaan.","Anehnya, dia bertemu dengan seorang pria tampa...","Setelah beberapa minggu, mereka bersama dalam ..."
...,...,...,...,...,...,...,...
933,6cb171ed-99f3-46f1-8d98-5dad776d9555,Wahyu Mike,Mike menunda meminta pacarnya untuk menikah de...,Dia telah mengatakan kepadanya bahwa dia ingin...,"Sekarang kuliah itu selesai, dia merasa tertek...","Sayangnya untuk Mike, keluar bahwa dia tidak i...",Sekarang Mike menyadari bahwa dia seharusnya j...
934,4de38ecd-39a3-4c7c-988e-93e6f8ec7705,Ultimatum Karen,Karen telah bersama pacarnya selama bertahun -...,Sekarang adik laki -lakinya bertunangan dan me...,Karen merasa sudah waktunya pacarnya untuk ber...,Dia memberinya ultimatum untuk menikah atau pe...,Pacarnya memilih untuk pergi.
935,7ce8f342-d590-40de-a312-aec4a638dcc9,Pernikahan atau kami putus,Janet dan John telah berkencan selama 10 tahun.,Janet selalu ingin menikah dengan John.,Tapi John tidak pernah ingin menikah.,Janet memberi tahu John bahwa mereka akan meni...,"Sayangnya untuk Janet, John memilih bahwa mere..."
936,31be3d5e-1c96-4c46-92e1-70bb1a888ee6,Bayi YouTube,Latoya dan Adam bertemu di acara YouTube.,Mereka berdua memiliki saluran YouTube sendiri.,Keduanya memiliki chemistry yang hebat dan mul...,Setahun kemudian mereka pindah bersama dan men...,Mereka telah bersama selama 5 tahun sekarang d...


In [6]:
tokenizer = AutoTokenizer.from_pretrained("indonesian-nlp/gpt2")
print("Sebelum")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

special_tokens_dict = {'additional_special_tokens': ['<|startoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print("Setelah")
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)  
print("Vocabulary size:", tokenizer.vocab_size)
print("Pad token ID:", tokenizer.pad_token_id)


Sebelum
['<|endoftext|>']
[50256]
Setelah
['<|endoftext|>', '<|startoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']
[50256, 50257, 50258, 50259, 50260, 50261]
Vocabulary size: 50257
Pad token ID: None


In [7]:
column_unchanged = ['storyid', 'storytitle']


# judul_token = '<|titletext|>'
start_token = '<|startoftext|> '
token_1to2 = ' <|sentence1to2|> '
token_2to3 = ' <|sentence2to3|> '
token_3to4 = ' <|sentence3to4|> '
token_4to5 = ' <|sentence4to5|> '
end_token = ' <|endoftext|>'


translated_dataset['MergedSentence'] = (
                start_token + 
                translated_dataset['sentence1'].astype(str) + 
                token_1to2 +
                translated_dataset['sentence2'].astype(str) + 
                token_2to3 + 
                translated_dataset['sentence3'].astype(str) + 
                token_3to4 +
                translated_dataset['sentence4'].astype(str) + 
                token_4to5 +
                translated_dataset['sentence5'].astype(str) + 
                end_token
                )

merged_translated_datasets = translated_dataset[column_unchanged + ['MergedSentence']]

print(merged_translated_datasets.head(1))



                                storyid storytitle  \
0  f3643541-c339-4377-851d-144ea400d551   Ibu tiri   

                                      MergedSentence  
0  <|startoftext|> Eric dan istrinya memiliki seo...  


In [5]:
# shuffled_dataset = merged_translated_datasets.sample(frac=1, random_state=42)

num_train_rows = int(len(translated_dataset) * 0.8)

translated_dataset_train = translated_dataset.iloc[:num_train_rows, :]
translated_dataset_test = translated_dataset.iloc[num_train_rows:, :]


In [6]:
with open('train_translated_romance_output.txt', 'w') as f:
    for text in translated_dataset_train['MergedSentence'].tolist():
        f.write(text + '\n')
with open('test_translated_romance_output.txt', 'w') as f:
    for text in translated_dataset_test['MergedSentence'].tolist():
        f.write(text + '\n')

In [7]:
max_length = translated_dataset_train['MergedSentence'].str.len().max()
print (max_length)

max_length = translated_dataset_test['MergedSentence'].str.len().max()
print (max_length)


564
547


In [8]:
from datasets import Dataset
print("Size of the training set:")
print("Rows:", translated_dataset_train.shape[0])
print("Columns:", translated_dataset_train.shape[1])

# Display the size of the testing set
print("\nSize of the testing set:")
print("Rows:", translated_dataset_test.shape[0])
print("Columns:", translated_dataset_test.shape[1])

Size of the training set:
Rows: 750
Columns: 8

Size of the testing set:
Rows: 188
Columns: 8


In [6]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)  

print("Vocabulary size:", tokenizer.vocab_size)
print("Pad token ID:", tokenizer.pad_token_id)

['<|endoftext|>', '<|startoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']
[50256, 50257, 50258, 50259, 50260, 50261]
Vocabulary size: 50257
Pad token ID: None


In [3]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
model = AutoModelWithLMHead.from_pretrained("indonesian-nlp/gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-ROCStories-romance-resumed", 
    overwrite_output_dir=True, 
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy='steps',
    eval_steps=1, 
    logging_steps=1,
    save_steps=200, 
    warmup_steps=300,
    prediction_loss_only=True
)

train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='train_translated_output.txt',
    block_size=560
)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='test_translated_output.txt',
    block_size=560
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

writer = SummaryWriter(f'./gpt2-ROCStories-romance-resumed/tensor_logs')
logger = TensorBoardCallback()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    callbacks=[logger], 
)


c:\Users\asus\anaconda3\envs\Transformers-pandas\lib\site-packages\torch\distributed\_functional_collectives.py:22: UserWarning: Unable to import torchdynamo util `is_torchdynamo_compiling`, so won't support torchdynamo correctly
  warnings.warn(


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
module 'torch' has no attribute '_utils'

In [2]:
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
model = AutoModelForCausalLM.from_pretrained("indonesian-nlp/gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-ROCStories-romance-resumed", 
    overwrite_output_dir=True, 
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy='steps',
    eval_steps=1, 
    logging_steps=1,
    save_steps=300, 
    warmup_steps=450,
    prediction_loss_only=True
)

train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='train_translated_romance_output.txt',
    block_size=570
)

test_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='test_translated_romance_output.txt',
    block_size=570
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

writer = SummaryWriter(f'./gpt2-ROCStories-romance-resumed/tensor_logs')
logger = TensorBoardCallback()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    callbacks=[logger], 
)


C:\Users\asus\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [7]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)  
print("Vocabulary size:", tokenizer.vocab_size)
print("Pad token ID:", tokenizer.pad_token_id)

['<|endoftext|>', '<|firstoftext|>', '<|lastoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']
[50256, 50257, 50258, 50259, 50260, 50261, 50262]
Vocabulary size: 50257
Pad token ID: None


In [1]:
trainer.train(resume_from_checkpoint=True)

NameError: name 'trainer' is not defined

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = "" 

In [1]:
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():

   print("CUDA Hardware Accelerator Activated")

else:

   print("CUDA Hardware Accelerator NOT Available")

False
CUDA Hardware Accelerator NOT Available


In [2]:
import torch
torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [1]:
import torch
torch.cuda.empty_cache()

In [1]:
import torch
import numpy as np
import time

flatten_masks = np.random.random((800, 60800))

flatten_masks = torch.from_numpy(flatten_masks).cuda(device=0)
print()
t1 = time.time()
i = 0
while i < 2500:
    if i == 500:
        t1 = time.time()
    # old version 
    inter_matrix = torch.mm(flatten_masks, flatten_masks.transpose(1, 0))
    # new version
    # inter_matrix = torch.einsum('ik, kj -> ij', flatten_masks, flatten_masks.transpose(1, 0))
    i += 1
t2 = time.time()
print(t2-t1)

KeyboardInterrupt: 

In [2]:
from torch.utils.tensorboard import SummaryWriter

In [1]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
from transformers import TextDataset,LineByLineTextDataset,DataCollatorForLanguageModeling
from optuna.visualization import plot_optimization_history, plot_param_importances
from transformers import AutoTokenizer
import optuna



def objective(trial):
    trial_learning_rate = trial.suggest_loguniform('learning_rate', 5e-5, 5e-1)
    trial_batch_size = trial.suggest_categorical('batch_size', [1])
    trial_eval_steps = trial.suggest_int('eval_steps', 1, 1000)
    trial_num_train_epochs = trial.suggest_int('num_train_epochs', 1, 1)
    trial_warmup_steps = trial.suggest_int('warmup_steps', 100, 1000)

    training_args = TrainingArguments(
        output_dir=f"./gpt2-ROCStories-optuna/{trial.number}", 
        overwrite_output_dir=True, 
        num_train_epochs=trial_num_train_epochs,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        evaluation_strategy='steps',
        eval_steps=trial_eval_steps, 
        logging_steps=300,
        save_steps=300, 
        warmup_steps=trial_warmup_steps,
        learning_rate=trial_learning_rate,
        prediction_loss_only=True
    )

    model = AutoModelForCausalLM.from_pretrained("indonesian-nlp/gpt2")
    tokenizer = AutoTokenizer.from_pretrained("indonesian-nlp/gpt2")
    print("Sebelum")
    print(tokenizer.all_special_tokens)
    print(tokenizer.all_special_ids)

    special_tokens_dict = {'additional_special_tokens': ['<|startoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']}
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    tokenizer.pad_token = tokenizer.eos_token
    # if tokenizer.pad_token is None:
    #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    print("Setelah")
    print(tokenizer.all_special_tokens)
    print(tokenizer.all_special_ids)  
    print("Vocabulary size:", tokenizer.vocab_size)
    print("Pad token ID:", tokenizer.pad_token_id)

    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path='train_translated_output.txt',
        block_size=560
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path='test_translated_output.txt',
        block_size=560
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )

    writer = SummaryWriter(f'./gpt2-ROCStories-optuna/tensor_logs/{trial.number}')
    logger = TensorBoardCallback()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=data_collator,
        callbacks=[logger], 
    )

    trainer.train()

    results = trainer.evaluate()
    return results["eval_loss"]

if __name__ == "__main__":
    study = optuna.create_study(direction='minimize') 
    study.optimize(objective, n_trials=10)

    best_params = study.best_params
    print(f"Best Hyperparameters: {best_params}")

    plot_optimization_history(study)

    plot_param_importances(study)



C:\Users\asus\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-12-31 02:08:39,323] A new study created in memory with name: no-name-6d3b5229-0e33-4edc-ade4-62ced88ee273
C:\Users\asus\AppData\Local\Temp\ipykernel_54924\3142086410.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  trial_learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-1)


Sebelum
['<|endoftext|>']
[50256]
Setelah
['<|endoftext|>', '<|startoftext|>', '<|sentence1to2|>', '<|sentence2to3|>', '<|sentence3to4|>', '<|sentence4to5|>']
[50256, 50257, 50258, 50259, 50260, 50261]
Vocabulary size: 50257
Pad token ID: 50256


c:\Users\asus\anaconda3\envs\Transformers-pandas\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
 12%|█▏        | 300/2586 [02:33<18:46,  2.03it/s]

{'loss': 2.3301, 'learning_rate': 0.010370811674772, 'epoch': 0.12}


                                                    
 15%|█▌        | 391/2586 [05:00<18:01,  2.03it/s]

...

{'eval_loss': 1.5958836078643799, 'eval_runtime': 87.5231, 'eval_samples_per_second': 7.21, 'eval_steps_per_second': 7.21, 'epoch': 0.95}


100%|██████████| 2586/2586 [51:06<00:00,  1.19s/it]


{'train_runtime': 3066.0117, 'train_samples_per_second': 0.843, 'train_steps_per_second': 0.843, 'train_loss': 1.677995037129012, 'epoch': 1.0}


100%|██████████| 631/631 [01:27<00:00,  7.18it/s]
[I 2023-12-31 08:12:44,067] Trial 9 finished with value: 1.5957720279693604 and parameters: {'learning_rate': 2.268308085086957e-05, 'batch_size': 1, 'eval_steps': 351, 'num_train_epochs': 1, 'warmup_steps': 207}. Best is trial 3 with value: 1.4995778799057007.


Best Hyperparameters: {'learning_rate': 0.0007947446194351545, 'batch_size': 1, 'eval_steps': 892, 'num_train_epochs': 1, 'warmup_steps': 363}


In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.all_special_tokens)
print(tokenizer.pad_token)

NameError: name 'tokenizer' is not defined

In [3]:

plot_optimization_history(study)


In [4]:

plot_param_importances(study)

In [3]:
from optuna.visualization import plot_slice
from optuna.visualization import plot_contour
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances

plot_slice(study)


In [5]:
plot_contour(study)



[W 2023-12-31 09:19:47,961] Param batch_size unique value length is less than 2.
[W 2023-12-31 09:19:47,962] Param batch_size unique value length is less than 2.
[W 2023-12-31 09:19:47,964] Param num_train_epochs unique value length is less than 2.
[W 2023-12-31 09:19:47,966] Param batch_size unique value length is less than 2.
[W 2023-12-31 09:19:47,968] Param batch_size unique value length is less than 2.
[W 2023-12-31 09:19:47,969] Param num_train_epochs unique value length is less than 2.
[W 2023-12-31 09:19:47,971] Param batch_size unique value length is less than 2.
[W 2023-12-31 09:19:47,972] Param num_train_epochs unique value length is less than 2.
[W 2023-12-31 09:19:47,973] Param batch_size unique value length is less than 2.
[W 2023-12-31 09:19:47,975] Param num_train_epochs unique value length is less than 2.
[W 2023-12-31 09:19:47,977] Param num_train_epochs unique value length is less than 2.
[W 2023-12-31 09:19:47,979] Param num_train_epochs unique value length is less 

In [6]:
plot_parallel_coordinate(study)


"<|startoftext|>", "Yeremia", "sangat", "suka", "pergi", "ke", "bioskop.", "<|sentence1to2|>", "Segala", "sesuatu", "tentang", "film", "yang", "dia", "sukai.", "<|sentence2to3|>", "Popcorn,", "trailer,", "semuanya", "adalah", "bagian", "dari", "pengalaman.", "<|sentence3to4|>", "Yeremia", "selalu", "senang", "pergi", "ke", "bioskop.", "<|sentence4to5|>", "Dia", "menyukai", "semuanya.", "<|endoftext|>"
